In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp  

def calculate_constant (mass, gravity, C_Drag, diameter,rho):
   a=np.array([0,gravity,0])
   b_const=-(C_Drag*rho)/(2*mass)
   b=np.array([b_const,b_const,b_const])
   return a,b

def ode_system(t, state, a, b):
    x, y, z, Vx, Vy, Vz = state
    dxdt = Vx
    dydt = Vy
    dzdt = Vz
    dVxdt = a[0] + b[0] * Vx**2
    dVydt = a[1] + b[1] * Vy**2
    dVzdt = a[2] + b[2] * Vz**2
    return [dxdt, dydt, dzdt, dVxdt, dVydt, dVzdt]

def check_collision(state, backpost_x, backpost_height_min, backpost_height_max):
    x, y = state[0], state[1]
    return backpost_x <= x <= backpost_x + 0.1 and backpost_height_min <= y <= backpost_height_max

def check_hit(Y_target,Y_tolerance,Z_target,Z_tolerance,Y_pos,X_pos,Z_pos): #Pos = posisi bola
   if np.isclose(Y_pos,Y_target,atol=Y_tolerance) and np.isclose(Z_pos,Z_target,atol=Z_tolerance):
         X_hit=X_pos
   return X_hit

def simulate(initial_conditions,debug_simulation,backpost_x, backpost_height_min, backpost_height_max,a,b,X_target,Y_target,Z_target,X_tol,Y_tol,Z_tol):
   if debug_simulation:
       x_values = []
       y_values = []
   time = 0
   X_hit=0
   last_state = []
   Isbounce=False
   while True:
      t_span = (time, time + 0.001)  # Simulate for 0.1 seconds
      sol = solve_ivp(ode_system, t_span, initial_conditions, args=(a, b), dense_output=True)
      t_eval = np.linspace(sol.t[0], sol.t[1], 1000)  #Evaluasi 100 titik
      state_eval = sol.sol(t_eval)
      X_hit = check_hit(Y_target,Y_tol,X_target,X_tol,Z_target,Z_tol,state_eval[1],state_eval[0],state_eval[2])
      Isbounce = check_collision(state_eval[:],backpost_x,backpost_height_min,backpost_height_max)

      #=========DEBUG MODE============#
      if debug_simulation:
         x_values.extend(state_eval[0])
         y_values.extend(state_eval[1])

      #========BREAKER================#
      if np.isclose(X_hit,X_target,X_tol):
         Score=1
         break
      
      elif Isbounce:
         Isbounce = True
         last_state=[state_eval[0,-1],state_eval[1,-1],state_eval[2,-1],state_eval[3,-1],state_eval[4,-1],state_eval[5,-1]]
         break
   
   return Isbounce,last_state,Score


def calculate_BestShot (SimData): # SimData = [V,Theta,phi,Score]
    V = SimData[:,0]
    Theta =  SimData [:,1]
    phi = SimData[:,2]
    Score =  SimData [:,3]

    
    return
##=================MAIN==============##
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp  


debugMain=True
#Parameter
Mass=0.600
Diameter=0.240
C_drag=0.54
rho=1.204
Gravity=9.81
ode_Constant = np.array(calculate_constant(Mass,Gravity,C_drag,Diameter,rho))
if debugMain:
    print("a= ",ode_Constant[0]," b = ",ode_Constant[1])


#==================Simulation function===========##
#Loop Variable
debugInitSim=True
R_maks = 10
R_min = 1
R_interval = 1

V_maks = 15
V_min = 0
V_interval = 1

Theta_maks = 80
Theta_min = 10
Theta_interval = 1

phi_maks = 80
phi_min = 10
phi_interval = 1

R_var=np.arange(R_min,R_maks,R_interval)
V_var=np.arange(V_min,V_maks,V_interval)
Theta_var=np.arange(Theta_min,Theta_maks,Theta_interval)
phi_var=np,arange(phi_min,phi_maks,phi_interval)


if debugInitSim:
    print(R_var,V_var,Theta_var)

#initial condition
X_shooter = 0
Y_shooter = 0.02
Z_shooter = 0
VX_robot = 0
VY_robot = 0
VZ_robot = 0

Initial_Condition = [X_shooter,Y_shooter,Z_shooter,VX_robot,VY_robot,VZ_robot]
if debugInitSim:
    print ("initial condition (X,Y,Z,VX,VY,VZ)\n",Initial_Condition)

#Toleransi
X_tolerance=0.11
Z_tolerance=0.11
Y_tolerance=0.02

SimData = []

##==============SIMULATION PROGRAM================##
for R in R_var:
    for V in V_var:
        for Theta in Theta_var:
            for phi in phi_var:
            #Iteration parameter
                VX_0 = V*np.cos(np.radians(phi))*np.sin(np.radians(Theta))
                VY_0 = V*np.sin(np.radians(phi))
                VZ_0 = V*np.cos(np.radians(phi))*np.cos(np.radians(Theta))

                #3D Object
                #Backpost
                hooptoBackpost = 0.24
                Backpost_X = R + hooptoBackpost
                Backpost_Y1 = 1.9
                Bacpost_Y2 = 2.9
                Backpost_Z1 = -0.9 
                Backpost_Z2 = 0.9

                #Target
                Target_X=R
                Target_Y=2.43
                Target_Z=0
                Sim_Result = np.array(simulate(Initial_Condition,False,Backpost_X,Backpost_Y1,Bacpost_Y2,ode_Constant[0],ode_Constant[1],Target_X,Target_Y,Target_Z,X_tolerance,Y_tolerance,Z_tolerance))
                #Sim_result [Isbounce,last_state,Score]

                IsBounce =  Sim_Result [0]
                Last_state = Sim_Result [1]
                Score = Sim_Result[2]

                if IsBounce:
                    Initial_Condition = Last_state
                    Sim_Result = np.array(simulate(Initial_Condition,False,Backpost_X,Backpost_Y1,Bacpost_Y2,ode_Constant[0],ode_Constant[1],Target_X,Target_Y,Target_Z,X_tolerance,Y_tolerance,Z_tolerance))
                    IsBounce = False
                    Score = Sim_Result[2]
                
                SimData.append(V,Theta,phi,Score)



                

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (2221570696.py, line 164)